In [1]:
import numpy as np
import matplotlib.pylab as plt
from pylab import *
import sys
import pandas as pd

In [2]:
##########################
# define a few constants #		##fft python random phase
##########################

e0 = np.loadtxt('837nm.dat')		#load the reference pulse, pump @836 nm
e0 = e0.T
e1 = np.sqrt(e0[1])
e1_ph = e0[2]
e = np.array(e1)*np.exp(-1j*(np.array(e1_ph)))		#reforming E-field

original_trace = np.loadtxt('trace/trace1.frg')		#load FROG trace from experiment

N = len(e0[0])		#timesteps
N_max = np.max(e0[0])
N_min = np.min(e0[0])

t = e0[0]-N_min
T0 = N_max-N_min	#total time duration
dt = T0/(N-1)
df = 1.0/T0		#frequency steps
TD = N			#time delay for FROG-trace generation
freq = np.arange(0,1/dt+df,df)*1000

In [3]:
original_trace.shape

(128, 128)

In [4]:
## for generation of Gaussian pulses
def gaussian_pulse(T,f,A,phase,tau):
	w0 = f*2*np.pi		#central angular frequency of pulse
	temp = -1j*w0*(t-tau) + 1j*phase
	E = A*np.sqrt(np.exp(-4.0*np.log(2.0)/np.log(np.e)*((t-tau)/T)**2))*np.exp(-temp)
	return E

In [5]:
## calculate cross correlation of two pulses
def xcorr(e0,e,td):		#shift one pulse by time delay of 'td'
	E_delay = e*1.0
	if td<=0:
		E_delay[0:N+td] = e0[-td:]
		E_delay[N+td:] = [0]*(-td)
	else:
		E_delay[0:td] = [0]*(td)
		E_delay[td:] = e0[0:N-td]	
	return  e*E_delay

In [6]:
## calculate the differert of trace A and trace B
def array_diff(A,B):
	A = A/np.max(A)
	B = B/np.max(B)
	return np.sum(np.square(A-B))/np.sum(np.square(A))

In [7]:
def FROG_generator(e,E,TD):	
	FROG = np.zeros((TD,N),dtype=float)
	for td in range(-TD//2,TD//2):
		E_corre = xcorr(e,E,td)
		temp0 = np.fft.fftshift(np.fft.fft(E_corre))
		FROG[(td+TD//2)-1] = abs(temp0)**2	
	return FROG.T

In [8]:
## plot out FROG trace
def FROG_trace_plot(FROG):
	fig,ax = plt.subplots()
	plt.imshow(FROG)
	plt.title("FROG trace")
	plt.grid(True)
	plt.show()

In [9]:
## calculate FROG traces with different time delays
def FROGs(T1,T2,A1,A2,f0,f1,tau1):
    tau2 = tau1-700
    phase1 = a2*(t-tau1)**2+a3*(t-tau1)**3
    phase2 = a2*(t-tau2)**2+a3*(t-tau2)**3 
    E1 = gaussian_pulse(T1,f0,A1,phase1,tau1)
    E2 = gaussian_pulse(T2,f1,A2,phase2,tau2)
    E = E1+E2
    FROG = FROG_generator(e,E,TD)
    delta = array_diff(FROG, original_trace)	
    return FROG, delta

In [10]:
######################
# initial parameters #
######################

T1 = np.arange(300,1000,10)		#Gaussian pulse duration in fs, FW
T2 = np.arange(300,1000,10)
A1 = np.arange(0.6,1.5,0.05)			#Amplitude of Raman pulse
A2 = np.arange(0.6,1.5,0.05)

a2 = 2.50*10**-6
#a2 = np.delete(a2, np.where(np.abs(a2) < 1e-10))#second order phase factor
a3 = 3.0*10**-9		#third order phase factor
#a3 = np.delete(a3, np.where(np.abs(a3) < 1e-14))

F0 = np.arange(-0.005, 0.005, 0.0005)
F0 = np.delete(F0, np.where(np.abs(F0) < 1e-10))#Raman center-wavelength
F1 = np.arange(-0.0005, 0.0005, 0.00005)
F1 = np.delete(F1, np.where(np.abs(F1) < 1e-10))#Red-shifted center-wavelength
			
i0 = 333.0		#unit: fs
Tau1 = np.arange(-1000, 5000, 50)	#Raman pulse peak time
#tau2 = tau1-700		#Red-shifted pulse peak time

In [11]:
Tau1

array([-1000,  -950,  -900,  -850,  -800,  -750,  -700,  -650,  -600,
        -550,  -500,  -450,  -400,  -350,  -300,  -250,  -200,  -150,
        -100,   -50,     0,    50,   100,   150,   200,   250,   300,
         350,   400,   450,   500,   550,   600,   650,   700,   750,
         800,   850,   900,   950,  1000,  1050,  1100,  1150,  1200,
        1250,  1300,  1350,  1400,  1450,  1500,  1550,  1600,  1650,
        1700,  1750,  1800,  1850,  1900,  1950,  2000,  2050,  2100,
        2150,  2200,  2250,  2300,  2350,  2400,  2450,  2500,  2550,
        2600,  2650,  2700,  2750,  2800,  2850,  2900,  2950,  3000,
        3050,  3100,  3150,  3200,  3250,  3300,  3350,  3400,  3450,
        3500,  3550,  3600,  3650,  3700,  3750,  3800,  3850,  3900,
        3950,  4000,  4050,  4100,  4150,  4200,  4250,  4300,  4350,
        4400,  4450,  4500,  4550,  4600,  4650,  4700,  4750,  4800,
        4850,  4900,  4950])

In [12]:
values = pd.DataFrame(columns = ['T1', 'T2', 'A1', 'A2', 'F0', 'F1', 'Tau1'])
values

,T1,T2,A1,A2,F0,F1,Tau1


In [13]:
###########################
# FROG trace main program #
###########################
FROGS = []
values = []
for i in range(5000):
    t1 = np.random.choice(T1)
    t2 = np.random.choice(T2)
    ai1 = np.random.choice(A1)
    ai2 = np.random.choice(A2)
    f0 = np.random.choice(F0)
    f1 = np.random.choice(F1)
    #aa2 = np.random.choice(a2)
    #aa3 = np.random.choice(a3)
    tau1 = np.random.choice(Tau1)
    (FROG, delta) = FROGs(t1,t2,ai1,ai2,f0,f1,tau1)
    print(delta)
    values.append([t1, t2, ai1, ai2, f0, f1, tau1])
    FROGS.append(FROG)

2.516142527525365
2.2112153979310385
2.4711675564834095
1.7785500187272267
2.6279948280897933
2.2274367285179504
2.353807386231977
2.1310762384752593
3.010484376102287
1.9181419316151505
2.269738386276748
3.083188967394177
2.2741698949910067
1.4804189132589973
1.7866820521091542
2.0958623874264517
2.214520951843437
2.8865683795728416
2.066485279181262
2.520723222167974
2.146644949943414
2.0327016682775945
1.3331404636818196
2.471350042011185
2.1338094260752327
2.444660220979671
2.6248240222907633
2.4822515608944213
2.686201235704498
1.7579678207828406
1.7636373986564438
3.0347171589806625
2.5971306613094125
2.483064034472112
1.4596232394421798
1.5020768494320984
2.32541129958613
1.4987384993076946
2.178537542815434
2.33826362545136
2.268269152664072
1.3729316389306663
2.4355635806381826
2.7607535094651454
1.4987104842427321
1.9419841998460532
1.8384821448429343
1.3017051029796136
2.6169763166049878
2.423389545298422
2.08073096001594
2.3702466707667584
2.5819837660255485
2.1468247829830

In [14]:
FROGS = np.array(FROGS)
#FROGS = FROGS.reshape(1000,-1)

In [16]:
values = np.array(values)
values.shape

(5000, 7)

In [17]:
np.save("FROG_test", FROGS)

In [19]:
np.save("labels_cnn", values)